# 🚀 Simple Model Debug - Train & Test

**Simple workflow:**
1. **Train** model on `/data/` (BTCUSDT)
2. **Test** model on `/data_test/` with plotly visualization

In [ ]:
# Imports
import pandas as pd
import numpy as np
import json
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from datetime import datetime

from src.autonomous_trader_trainer import AutonomousTraderTrainer
from src.autonomous_trader import AutonomousTrader

pio.templates.default = "plotly_dark"
print("✅ Imports ready")

In [ ]:
# ================================================
# 🎓 TRAIN MODEL (using /data/ BTCUSDT)
# ================================================

from src.simple_model_trainer import SimpleModelTrainer

print("🎓 TRAINING MODEL ON BTCUSDT DATA")
print("=" * 50)

# Training files
training_files = {
    '15m': 'data/BTCUSDT-15m.json',
    '1h': 'data/BTCUSDT-1h.json',
    'M': 'data/BTCUSDT-M.json',
    'W': 'data/BTCUSDT-W.json', 
    'D': 'data/BTCUSDT-D.json'
}

# Check files exist
for tf, path in training_files.items():
    exists = "✅" if os.path.exists(path) else "❌"
    print(f"   {tf}: {path} {exists}")

# Initialize trainer
trainer = SimpleModelTrainer()

# Configure training parameters
trainer.configure_training(
    profit_threshold=2.0,
    loss_threshold=-1.5,
    lookforward_periods=[5, 10, 20]
)

print(f"\n📊 Training Configuration:")
print(f"   Profit threshold: {trainer.profit_threshold}%")
print(f"   Loss threshold: {trainer.loss_threshold}%")
print(f"   Lookforward periods: {trainer.lookforward_periods}")

# Train the model
success = trainer.train_model(
    training_files=training_files,
    level_timeframes=['M', 'W', 'D', '1h']
)

if success:
    # Get model info
    info = trainer.get_model_info()
    print(f"\n🎉 TRAINING SUCCESS!")
    print(f"   Model Type: {info['model_type']}")
    print(f"   Accuracy: {info['accuracy']:.1%}")
    print(f"   Features: {info['features']}")
    print(f"   Classes: {info['classes']}")
else:
    print("❌ Training failed!")
    trainer = None

In [ ]:
# ================================================
# 🧪 TEST MODEL (using /data_test/)
# ================================================

from src.simple_model_trainer import SimpleModelPredictor

print("🧪 TESTING MODEL ON UNSEEN DATA")
print("=" * 50)

# Load trained model if not already loaded
if 'trainer' not in locals() or trainer is None:
    trainer = SimpleModelTrainer()
    if not trainer.load_model():
        print("❌ No trained model found! Please run the training cell first.")
        exit()

# Initialize predictor
predictor = SimpleModelPredictor(trainer)

# Test configuration
TEST_SYMBOL = 'BTCUSDT'  # Change this to test different symbols
NUM_CANDLES = 200        # Number of recent candles to analyze

print(f"🎯 Testing on {TEST_SYMBOL} (last {NUM_CANDLES} candles)")

# Generate predictions
signals_df = predictor.generate_predictions(
    symbol=TEST_SYMBOL,
    num_candles=NUM_CANDLES,
    data_folder='data_test'
)

if signals_df is not None:
    # CREATE PLOTLY CHART
    print(f"\n📈 Creating interactive chart...")
    
    model_info = trainer.get_model_info()
    
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=(f'{TEST_SYMBOL} - {model_info["model_type"]} Predictions', 'Confidence Score'),
        vertical_spacing=0.1,
        row_heights=[0.7, 0.3]
    )
    
    # Candlestick chart
    fig.add_trace(
        go.Candlestick(
            x=signals_df['datetime'],
            open=signals_df['open'],
            high=signals_df['high'],
            low=signals_df['low'],
            close=signals_df['close'],
            name='Price',
            showlegend=True
        ),
        row=1, col=1
    )
    
    # BUY signals
    buy_signals = signals_df[signals_df['action'] == 'buy']
    if not buy_signals.empty:
        fig.add_trace(
            go.Scatter(
                x=buy_signals['datetime'],
                y=buy_signals['close'],
                mode='markers',
                marker=dict(symbol='triangle-up', size=15, color='lime'),
                name='BUY Prediction',
                text=buy_signals['reasoning'],
                hovertemplate='<b>BUY</b><br>Price: $%{y:.4f}<br>Confidence: %{customdata:.1%}<br>%{text}<extra></extra>',
                customdata=buy_signals['confidence']
            ),
            row=1, col=1
        )
    
    # SELL signals
    sell_signals = signals_df[signals_df['action'] == 'sell']
    if not sell_signals.empty:
        fig.add_trace(
            go.Scatter(
                x=sell_signals['datetime'],
                y=sell_signals['close'],
                mode='markers',
                marker=dict(symbol='triangle-down', size=15, color='red'),
                name='SELL Prediction',
                text=sell_signals['reasoning'],
                hovertemplate='<b>SELL</b><br>Price: $%{y:.4f}<br>Confidence: %{customdata:.1%}<br>%{text}<extra></extra>',
                customdata=sell_signals['confidence']
            ),
            row=1, col=1
        )
    
    # HOLD signals
    hold_signals = signals_df[signals_df['action'] == 'hold']
    if not hold_signals.empty:
        fig.add_trace(
            go.Scatter(
                x=hold_signals['datetime'],
                y=hold_signals['close'],
                mode='markers',
                marker=dict(symbol='circle', size=8, color='gray'),
                name='HOLD Prediction',
                text=hold_signals['reasoning'],
                hovertemplate='<b>HOLD</b><br>Price: $%{y:.4f}<br>Confidence: %{customdata:.1%}<br>%{text}<extra></extra>',
                customdata=hold_signals['confidence']
            ),
            row=1, col=1
        )
    
    # Confidence line
    confidence_colors = ['lime' if action == 'buy' else 'red' if action == 'sell' else 'gray' 
                       for action in signals_df['action']]
    
    fig.add_trace(
        go.Scatter(
            x=signals_df['datetime'],
            y=signals_df['confidence'],
            mode='markers+lines',
            name='Model Confidence',
            line=dict(color='white', width=1),
            marker=dict(color=confidence_colors, size=8),
            hovertemplate='Confidence: %{y:.1%}<br>Prediction: %{customdata}<extra></extra>',
            customdata=signals_df['action']
        ),
        row=2, col=1
    )
    
    # Update layout
    fig.update_layout(
        title=f'🤖 {model_info["model_type"]} Predictions on {TEST_SYMBOL}',
        height=700,
        template='plotly_dark',
        showlegend=True
    )
    
    fig.update_yaxes(title_text="Price ($)", row=1, col=1)
    fig.update_yaxes(title_text="Confidence", row=2, col=1, tickformat='.0%')
    fig.update_xaxes(title_text="Time", row=2, col=1)
    
    fig.show()
    
    # Show high-confidence predictions
    high_conf = signals_df[signals_df['confidence'] > 0.8]
    if not high_conf.empty:
        print(f"\n🚀 High Confidence Predictions (>80%):")
        for _, pred in high_conf.head(5).iterrows():
            print(f"   {pred['datetime'].strftime('%Y-%m-%d %H:%M')}: {pred['action'].upper()} @ ${pred['close']:.4f} ({pred['confidence']:.1%})")
    
    print(f"\n✅ TESTING COMPLETE!")
    print(f"🎯 Model: {model_info['model_type']} (Accuracy: {model_info['accuracy']:.1%})")
    print(f"📊 Generated {len(signals_df)} predictions on {TEST_SYMBOL}")
else:
    print("❌ Failed to generate predictions")

🧪 TESTING MODEL ON UNSEEN DATA
📁 Loading trained model from: src/models/simple_trading_model.joblib
✅ Model loaded successfully!
   Model Type: XGBoost-GPU
   Accuracy: 95.3%
   Classes: ['buy', 'hold', 'sell']
   Features: 50

📁 Test files for BTCUSDT:
   15m: data_test/BTCUSDT-15m.json
   1h: data_test/BTCUSDT-1h.json
   D: data_test/BTCUSDT-D.json
   W: data_test/BTCUSDT-W.json
   M: data_test/BTCUSDT-M.json
✅ Loaded 87891 test candles (15m timeframe)
📊 Extracting levels from 3 timeframes...
📊 Available feature timeframes: ['1h', 'D', 'W', 'M']
🔄 Updating multi-timeframe levels...
Volume Profile: Generated 867 volume profile ranges
Volume Profile: Generated 867 volume profile ranges
✅ Extracted 953 levels from D timeframe
  Period 2020-03: SKIPPED - insufficient data
Volume Profile: Generated 198 volume profile ranges
✅ Extracted 238 levels from W timeframe
Volume Profile: Generated 18 volume profile ranges
✅ Extracted 35 levels from M timeframe
✅ Updated 1226 levels across 3 timefr


✅ TESTING COMPLETE!
🎯 XGBoost-GPU trained on BTCUSDT-15m → Predictions on BTCUSDT-15m
📊 Generated 200 predictions
🎲 Model accuracy: 95.3%
🔥 Average confidence: 99.1%

🚀 High Confidence Predictions (>80%):
   2025-10-03 00:30: HOLD @ $120139.0000 (99.2%)
   2025-10-03 00:45: HOLD @ $120090.1000 (98.9%)
   2025-10-03 01:00: HOLD @ $120172.1000 (99.1%)
   2025-10-03 01:15: HOLD @ $120050.7000 (99.2%)
   2025-10-03 01:30: HOLD @ $120027.5000 (99.2%)


## 🎉 Simple Model Debug - Results Summary

**What we just did:**
1. **Trained** XGBoost model with GPU/CPU comparison on BTCUSDT data
2. **Tested** the model on unseen data with interactive visualization
3. **Used clean, reusable classes** for all the heavy lifting

**Key improvements:**
- ✅ **Clean notebook**: Complex logic moved to `SimpleModelTrainer` class
- ✅ **Reusable code**: Training and prediction logic can be used anywhere
- ✅ **GPU acceleration**: Automatic GPU/CPU comparison and selection
- ✅ **Simple configuration**: Easy to change parameters and test symbols

**Architecture:**
- `SimpleModelTrainer`: Handles data prep, training, GPU/CPU comparison, model saving
- `SimpleModelPredictor`: Handles predictions, feature engineering, visualization data

**To test other symbols:**
Change `TEST_SYMBOL = 'BTCUSDT'` to `'ADAUSDT'`, `'DOGEUSDT'`, etc.

**To adjust training:**
Modify the `configure_training()` parameters in the training cell.